In [1]:

# External Dependencies
import os                          # Access system file-tree
import sys                         # Modify system parameters
from math import isclose           # isclose()
import numpy as np                 # Data structures
from optparse import OptionParser  # Get user input
import matplotlib.ticker as mticker  # Scientific notation in labels
from matplotlib.ticker import FuncFormatter

np.set_printoptions(threshold=sys.maxsize, precision=16)

In [2]:
# Internal Dependencies
internalPath = os.getcwd() + "/../python"
print(internalPath)
sys.path.insert(0, internalPath)
from plotStyling import PlotStyling  # noqa: E402
from GSDUtil import GSDUtil  # noqa: E402

/home/aglisman/Dropbox/3_Brady_Lab/Code/potential-flow/bodies-in-potential-flow/jupyter_notebooks/../python


In [3]:
# Parameters
relative_path = os.getcwd() + \
    "/../data_tag/v1.0.0/2021-08-19.14-24-14_collinear-swimmer_varyRelDisp"
output_dir = "figures"
output_dir = relative_path + "/" + output_dir + "/"
gsd_files = []
epsOutput = True


# Correctly get scientific notation in text elements
def scientific(x, pos):
    return '%0.2e' % x


scientific_formatter = FuncFormatter(scientific)
fmt = mticker.FuncFormatter(scientific_formatter)


In [4]:
# Load data from GSD

try:
    # Loop through all subdirectories in the 'data' directory
    for root, dirs, files in os.walk(relative_path, topdown=True):
        dirs.sort()
        files.sort()

        for file in files:  # Loop through all files in given directory
            if (".gsd" in file):
                with open(root + "/" + file) as g:
                    cur_gsd = GSDUtil(g.name, create_gsd=False)
                    gsd_files.append(cur_gsd)

    assert(len(gsd_files) > 0)

except:  # No files found
    raise IOError(
        f"Failure to load data. No files found in relPath {relative_path}")

In [7]:
# Load data into np structures

CoM_disp = np.zeros(len(gsd_files), dtype=np.double)
R_avg = np.zeros_like(CoM_disp, dtype=np.double)
Z_height = np.zeros_like(CoM_disp, dtype=np.double)
phaseShift = np.zeros_like(CoM_disp, dtype=np.double)
U0 = np.zeros_like(CoM_disp, dtype=np.double)
omega = np.zeros_like(CoM_disp, dtype=np.double)
epsilon = np.zeros_like(CoM_disp, dtype=np.double)
final_t = np.zeros_like(CoM_disp, dtype=np.double)

for i in range(len(gsd_files)):

    # Data from final frame
    gsd_files[i].snapshot = gsd_files[i].trajectory.read_frame(
        gsd_files[i].trajectory.file.nframes - 1)
    final_t[i] = gsd_files[i].snapshot.log['integrator/t']
    CoM_disp_comp = gsd_files[i].snapshot.log['particles/double_position'][1]

    # Data from initial frame (not 0)
    gsd_files[i].snapshot = gsd_files[i].trajectory.read_frame(1)
    R_avg[i] = gsd_files[i].snapshot.log['swimmer/R_avg']
    Z_height[i] = gsd_files[i].snapshot.log['swimmer/Z_height']
    phaseShift[i] = gsd_files[i].snapshot.log['swimmer/phase_shift']
    U0[i] = gsd_files[i].snapshot.log['swimmer/U0']
    omega[i] = gsd_files[i].snapshot.log['swimmer/omega']
    epsilon[i] = U0[i] / R_avg[i] / omega[i]
    CoM_disp_comp -= gsd_files[i].snapshot.log['particles/double_position'][1]

    CoM_disp[i] = np.linalg.norm(CoM_disp_comp)

In [8]:
# NOTE: Assuming each GSD has same number of frames
nframes = gsd_files[0].trajectory.file.nframes

# ANCHOR: Orientation vector (not unit norm)
q = np.zeros((len(gsd_files), 3, nframes - 2), dtype=np.double)
q0 = np.zeros((len(gsd_files), 3), dtype=np.double)

# ANCHOR: temporal data
time = np.zeros((len(gsd_files), nframes - 2), dtype=np.double)

# ANCHOR: kinematic data (simulation_number, particle_number, dimension_number, frame_number)
positions = np.zeros((len(gsd_files), 6, 3, nframes - 2), dtype=np.double)
velocities = np.zeros_like(positions, dtype=np.double)
accelerations = np.zeros_like(positions, dtype=np.double)

In [9]:
for i in range(len(gsd_files)):  # loop over all GSD files

    gsd_current = gsd_files[i]
    q_current = np.zeros((3, nframes - 2), dtype=np.double)
    q_current_norm = np.zeros((nframes - 2), dtype=np.double)

    for j in range(1, nframes - 1):  # loop over all frames in GSD file

        snapshot_current = gsd_current.trajectory.read_frame(j)

        q_current[:, j - 1] = snapshot_current.log['particles/double_position'][0]
        q_current[:, j - 1] -= snapshot_current.log['particles/double_position'][2]

        time[i, j - 1] = snapshot_current.log['integrator/t']

        positions[i, :, :, j -
                  1] = snapshot_current.log['particles/double_position']
        velocities[i, :, :, j -
                   1] = snapshot_current.log['particles/double_velocity']
        accelerations[i, :, :,  -
                      1] = snapshot_current.log['particles/double_moment_inertia']

    q[i, :, :] = q_current

q0 = q[:, :, 0]

In [10]:
q0 = q[:, :, 0]
q0_norm = np.linalg.norm(q0, axis=1)
q0 = np.divide(q0, q0_norm[:, np.newaxis])

print(q0)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [11]:
# ANCHOR: Angular orientational displacement
theta = np.zeros((len(gsd_files), nframes - 2), dtype=np.double)
theta_norm = np.zeros((nframes - 2), dtype=np.double)

for i in range(len(gsd_files)):

    theta[i, :] = np.dot(q0[i, :].reshape(1, 3), q[i, :, :])

    print(np.array(theta[i, 0] - theta[i, -1], dtype=np.double))

    theta_norm = np.linalg.norm(q0[i, :]) * np.linalg.norm(q[i, :, :], axis=0)
    theta[i, :] = theta[i, :] / theta_norm

print(np.array(theta[:, 0] - theta[:, -1], dtype=np.double))

0.16874625556592449
0.1687462555657575
0.16874625555698053
0.16874625554228295
0.16874625557531608
0.16874625555073308
0.1687462555857877
0.1687462555904471
0.16874625554712352
0.16874625557140988
0.1687462555512873
0.16874625559836076
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
# relative separation between particle pairs
R_loc = positions[:, 1, :, :]
R_loc_init = positions[:, 1, :, 0]
DR_loc = R_loc - R_loc_init[:, :, np.newaxis]
R_12 = positions[:, 0, :, :] - R_loc
R_32 = positions[:, 2, :, :] - R_loc
# distance between particle pairs
Dr_Loc = np.linalg.norm(DR_loc, axis=1)
r_12 = np.linalg.norm(R_12, axis=1)
r_32 = np.linalg.norm(R_32, axis=1)

print(r_12[0, 0]-r_12[0, -1])
print(time[0, -1]-24.0)

0.15643459134265347
0.9750000040030216
